In [1]:
import os
import glob
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly
import lightly.utils.io as io
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from PIL import Image
import numpy as np
import json

ModuleNotFoundError: No module named 'pytorch_lightning'

In [2]:
embeddings_path = '/home/jcejudo/UI_image_classification/visual_recom/checkpoints/embeddings_3000.csv'
model_path = '/home/jcejudo/UI_image_classification/visual_recom/checkpoints/model_3000.pth'

In [3]:
#load embeddings
embeddings, labels, filenames = io.load_embeddings(embeddings_path)

#fit nearest neighbours
nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(embeddings)
distances, indices = nbrs.kneighbors(embeddings)

#load model
batch_size = 64
num_workers = 2
num_ftrs = 32
input_size = 128

collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=input_size,
    vf_prob=0.5,
    rr_prob=0.5,
    cj_prob=0.0,
    random_gray_scale=0.0
)
dataset_train_simclr = lightly.data.LightlyDataset(
input_dir=data_path
)

dataloader_train_simclr = torch.utils.data.DataLoader(
dataset_train_simclr,
batch_size=batch_size,
shuffle=True,
collate_fn=collate_fn,
drop_last=True,
num_workers=num_workers
)

resnet = lightly.models.ResNetGenerator('resnet-18')
last_conv_channels = list(resnet.children())[-1].in_features
backbone = nn.Sequential(
    *list(resnet.children())[:-1],
    nn.Conv2d(last_conv_channels, num_ftrs, 1),
    nn.AdaptiveAvgPool2d(1)
)

model = lightly.models.SimCLR(backbone, num_ftrs=num_ftrs)
criterion = lightly.loss.NTXentLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
encoder = lightly.embedding.SelfSupervisedEmbedding(
    model,
    criterion,
    optimizer,
    dataloader_train_simclr

)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = encoder.to(device)
if device == 'cpu':
    encoder.model.load_state_dict(torch.load(model_path,map_location=device))
else:
    encoder.model.load_state_dict(torch.load(model_path))
encoder.model.eval()

NameError: name 'io' is not defined